In [1]:
# voila_chatbot_ui.py
import json
import random
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import ipywidgets as widgets
from IPython.display import display

# Load models and resources
model_lstm = tf.keras.models.load_model("lstm_model.keras")
model_gru = tf.keras.models.load_model("gru_model.keras")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
with open("label_encoder.pkl", "rb") as f:
    lbl_encoder = pickle.load(f)
with open("intents_1000_tags.json", "r", encoding="utf-8") as f:
    data = json.load(f)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))
classes = sorted(set(intent["tag"] for intent in data["intents"]))

def preprocess_input(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    sequence = tokenizer.texts_to_sequences([" ".join(tokens)])
    return pad_sequences(sequence, maxlen=tokenizer.num_words, padding="post")

def predict_class(text, model):
    padded = preprocess_input(text)
    probs = model.predict(padded)[0]
    threshold = 0.5
    return [{"intent": classes[i], "probability": str(prob)} for i, prob in enumerate(probs) if prob > threshold]

def get_response(intents_list):
    tag = intents_list[0]['intent']
    for intent in data["intents"]:
        if intent["tag"] == tag:
            return random.choice(intent["responses"])
    return "I'm not sure how to respond to that."

def chatbot_response(text, model_type):
    model = model_lstm if model_type == "LSTM" else model_gru
    intents = predict_class(text, model)
    return get_response(intents) if intents else "Sorry, I didn't get that."

chat_output = widgets.Output()
chat_input = widgets.Text(placeholder="Type your message here...", layout=widgets.Layout(width="70%"))
send_button = widgets.Button(description="Send", layout=widgets.Layout(width="25%"))
model_selector = widgets.ToggleButtons(options=["LSTM", "GRU"], value="LSTM", description="Model:", button_style="info")

def on_send_click(_):
    with chat_output:
        text = chat_input.value.strip()
        if text:
            chat_input.value = ""
            chat_output.append_stdout(f"You: {text}\n")
            response = chatbot_response(text, model_selector.value)
            chat_output.append_stdout(f"Bot: {response}\n\n")

send_button.on_click(on_send_click)
display(widgets.VBox([model_selector, chat_output, widgets.HBox([chat_input, send_button])]))
